# Data Preparation

1. **Setup**: installations, import libraries and create functions for data preparation

2. **Load Data**: load data from this dataset

3. **Clean Data**: Overview of data in this dataset; investigating and treating missing data, duplicate data and invalid data for other notebooks to prepare recommendation systems.  

## 1. Setup

In [1]:
#pip install langdetect

In [2]:
#pip install -U spacy

In [3]:
#pip install googletrans

In [5]:
#install spacy and download en in the jupyter notebook:
#import sys
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en

In [4]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import pearsonr
import string
import re
import googletrans
from google_trans_new import google_translator 
from googletrans import Translator
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [6]:
#Load data
def load_file(df): 
    """
    Reads csv file as a Pandas dataframe.
    
    Parameters
    ----------
    df: name of csv file 
    
    Returns 
    -------
    Dataframe of csv file
    
    ---
    """
    return pd.read_csv(df)

In [7]:
#Update dataframes: genome_scores_df, ratings_df, tags_df
def update_df(df):
    """
    Updates dataframes containing 'movieId' information.
    
    Parameters
    ----------
    df: name of dataframe
    'duplicate_titles_update': dataframe that contains movieIds that match to the same movie title.
    
    Returns
    -------
    Returns an updated dataframe.
    
    ---
    """
    cols = df.columns
    df = df.merge(duplicate_titles_update, how='left', on = 'movieId')
    df['updated_movieId'].fillna(df['movieId'],inplace=True)
    df['updated_movieId'] = df['updated_movieId'].astype('int64')
    df.drop('movieId', axis=1, inplace=True)
    df = df.rename(columns={'updated_movieId':'movieId'})
    df = df[cols]
    return df

## 2. Load data

In [8]:
#Load data and define dataframes
genome_tags = load_file("genome-tags.csv")
genome_scores = load_file("genome-scores.csv")
#links = load_file("links.csv")
movies = load_file("movies.csv")
ratings = load_file("ratings.csv")
tags = load_file("tags.csv")

#put all dataframes together
dataframes = [genome_tags,genome_scores,#links,
              movies,ratings,tags]

## 3. Clean Data

In [9]:
#Quick scan of each dataframe
for dataframe in dataframes: 
    print(dataframe.info())
    display(dataframe.head())
    display(dataframe.shape)
    print('Missing data: ', dataframe.isnull().sum())
    print('No. of duplicates', dataframe.duplicated().sum())
    print('No. of uniques', dataframe.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB
None


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


(1128, 2)

Missing data:  tagId    0
tag      0
dtype: int64
No. of duplicates 0
No. of uniques tagId    1128
tag      1128
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14862528 entries, 0 to 14862527
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 340.2 MB
None


,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


...

(27753444, 4)

Missing data:  userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
No. of duplicates 0
No. of uniques userId         283228
movieId         53889
rating             10
timestamp    22131556
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108997 non-null  int64 
 1   movieId    1108997 non-null  int64 
 2   tag        1108981 non-null  object
 3   timestamp  1108997 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.8+ MB
None


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


(1108997, 4)

Missing data:  userId        0
movieId       0
tag          16
timestamp     0
dtype: int64
No. of duplicates 0
No. of uniques userId        19325
movieId       45981
tag           74714
timestamp    911869
dtype: int64


**Findings** 
* 'links' is not relevant to build our recommendation systems 
* 'genome_scores': number of unique movieIds and tags equals to the number of entries in the dataframe
* 'movies': 58,098 movieIds but 58,020 movie titles - 78 duplicate titles exist and will be investigated further
* 'ratings' and 'tags': timestamps are irrelevant - will drop timestamps.
* 'tags' column of the 'tags' data has 16 missing tags - will let them be.  

In [10]:
#Number of movies in 'movies'
print('Number of movies in movies dataframe: ', movies['movieId'].nunique())

#Number of movies in 'ratings'
print('Number of movies in ratings: ', ratings['movieId'].nunique())
print('Number of movies in genome_scores: ', genome_scores['movieId'].nunique())
print('Number of movies in tags: ', tags['movieId'].nunique())

Number of movies in movies dataframe:  58098
Number of movies in ratings:  53889
Number of movies in genome_scores:  13176
Number of movies in tags:  45981


**Findings** 
* Not all movies are in the movies dataframe - 'movies' dataframe has the most comprehensive number of movies.
* Data limitation as movie recommendations are dependent on the dataframes used. 

In [11]:
#Remove links dataframe
#del links

### Movies Data

* Clean duplicates in the dataframe
* Clean 'title' column 

In [12]:
#Order dataframe by movieId and then title
movies = movies.sort_values(by='movieId')

In [13]:
#Split '|' in the 'genres' column 
movies['genres'] = movies['genres'].str.split('|')

#### Duplicates
Find and treat duplicates  

In [14]:
#Create dataframe of movie title duplicates 
duplicate_movies = movies[movies['title'].duplicated()]
duplicate_movies

,movieId,title,genres
9142,26958,Emma (1996),[Romance]
9157,26982,Men with Guns (1997),[Drama]
13309,64997,War of the Worlds (2005),"[Action, Sci-Fi]"
13395,65665,Hamlet (2000),[Drama]
13614,67459,Chaos (2005),"[Crime, Drama, Horror]"
...,...,...,...
57269,191775,Berlin Calling (2008),"[Comedy, Drama]"
57305,191867,Let There Be Light (2017),[Documentary]
57361,192003,Journey to the Center of the Earth (2008),"[Action, Adventure, Fantasy, Sci-Fi]"
57463,192243,Contact (1992),"[Drama, Horror, Mystery, Thriller]"


In [15]:
#Create duplicate_movie_titles containing movie titles that have duplicates in 'movies'
duplicate_movie_titles = duplicate_movies['title'].reset_index(drop=True)
#Create 'duplicate_movie_titles': an ordered series of movie titles 
duplicate_movie_titles = duplicate_movie_titles.sort_values()
#Preview head & tail of 'duplicate_movie_titles'
duplicate_movie_titles

9     20,000 Leagues Under the Sea (1997)
36                      Absolution (2015)
8                        Aftermath (2012)
10                         Aladdin (1992)
38                   Another World (2014)
                     ...                 
54                        The Void (2016)
31                           Truth (2015)
58                        Veronica (2017)
2                War of the Worlds (2005)
63                         Weekend (2011)
Name: title, Length: 78, dtype: object

In [16]:
#Create 'duplicate_titles': dataframe containing movies and their duplicates 
duplicate_titles = movies[movies['title'].isin(duplicate_movie_titles)]
duplicate_titles = duplicate_titles.sort_values(by=['title','movieId'])

#Show head & tail of 'duplicate_titles' 
display(duplicate_titles.head(20))

#Print shape of 'duplicate_titles' 
print(duplicate_titles.shape)

,movieId,title,genres
21121,102190,"20,000 Leagues Under the Sea (1997)","[Adventure, Romance, Sci-Fi]"
24626,114130,"20,000 Leagues Under the Sea (1997)","[Romance, Sci-Fi]"
33067,136696,Absolution (2015),"[Action, Adventure, Crime, Thriller]"
39377,151797,Absolution (2015),"[Drama, Thriller]"
20141,98485,Aftermath (2012),"[Drama, Thriller]"
24599,114040,Aftermath (2012),"[Action, Thriller]"
582,588,Aladdin (1992),"[Adventure, Animation, Children, Comedy, Musical]"
24657,114240,Aladdin (1992),"[Adventure, Animation, Children, Comedy, Fantasy]"
34711,140633,Another World (2014),[Documentary]
41491,156686,Another World (2014),[(no genres listed)]


(156, 3)


**Findings**
* Notice differences in 'genres' for each title - some are subsets, while others have slightly different genres.

**Next steps**

Create a dataframe showing the movieId, the duplicate movie title's movieId, and combine the list of genres: 

1. Create dataframe to store movie titles with duplicate movieIds 
2. Different genres between duplicates - merge and list of combined unqiue genres between each movie title and its duplicate
3. If the list of genres contains '(no genres listed)' and one or more genres, then remove '(no genres listed)'
4. Update genres of move titles with duplicates and remove duplicates from Movies dataframe. 

##### Dataframe containing duplicate movie titles

In [17]:
#(Unique) Movie titles in order contained in duplicate_titles 
duplicate_title = duplicate_titles['title'][0::2].reset_index(drop=True)

In [18]:
#Indexes for each movie title without its duplicate in order contained in duplicate_titles
duplicate_titles_index = duplicate_titles.index[0::2]

In [19]:
#MovieIds

#MovieIds for each movie title in order contained in duplicate_titles 
duplicate_movieId1 = duplicate_titles['movieId'][0::2].reset_index(drop=True)

#Corresponding MovieIds of each movie title's duplicate in order contained in duplicate_titles 
duplicate_movieId2 = duplicate_titles['movieId'][1::2].reset_index(drop=True)

In [20]:
#Genres

#Genres for each movie title in order contained in duplicate_titles 
duplicate_titles1 = duplicate_titles['genres'][0::2].reset_index(drop=True)

#Corresponding genres of each movie title's duplicate in order contained in duplicate_titles 
duplicate_titles2 = duplicate_titles['genres'][1::2].reset_index(drop=True)

#Combine genres for each movie title and their duplicates
duplicate_titles_genres = duplicate_titles1 + duplicate_titles2

#Collect unique genres for each movie title, as combined genre lists contain duplicate/overlapping genres 
for i in range(len(duplicate_titles_genres)): 
    duplicate_titles_genres[i] = list(set(duplicate_titles_genres[i]))

In [21]:
#Create duplicate_titles: dataframe of movie titles with duplicates by 
# - their movieId;
# - their duplicate movieId;
# - title name; and 
# - combined list of unique genres

duplicate_titles = {'movieId': duplicate_movieId1, 
                    'duplicate_movieId': duplicate_movieId2, 
                    'title': duplicate_title, 
                    'updated_genres': duplicate_titles_genres}

duplicate_titles = pd.DataFrame(duplicate_titles)

#Show head & tail of 'duplicate_titles'
duplicate_titles.head(20)

,movieId,duplicate_movieId,title,updated_genres
0,102190,114130,"20,000 Leagues Under the Sea (1997)","[Sci-Fi, Romance, Adventure]"
1,136696,151797,Absolution (2015),"[Adventure, Crime, Drama, Action, Thriller]"
2,98485,114040,Aftermath (2012),"[Action, Drama, Thriller]"
3,588,114240,Aladdin (1992),"[Adventure, Animation, Fantasy, Comedy, Childr..."
4,140633,156686,Another World (2014),"[(no genres listed), Documentary]"
5,162618,181675,Apparition (2014),"[Horror, (no genres listed), Thriller]"
6,174701,184893,Ava (2017),"[(no genres listed), Drama]"
7,104035,115777,Beneath (2013),[Horror]
8,66511,191775,Berlin Calling (2008),"[Drama, Comedy]"
9,130388,138656,Black Field (2009),"[(no genres listed), Romance, Drama]"


**Findings**
* Some movies contain other genres with '(no genres listed)'. For example, Another World (2014).

**Next steps** 
1.  Update genres with 'updated_genres'
2. Remove duplicates from 'movies' 
3. Remove '(no genres listed)' for movie genres containing these findings. (Note this does not include for movies with '(no genres listed)' as only the genre.)

##### Update genres and removing duplicates in 'movies' dataframe

In [22]:
#Remove '(no genres listed)' for movie genres containing these findings. 
#Note this does not include for movies with '(no genres listed)' as only the genre.

#Create empty list to store calculations that determine which rows need updating
no_genres_listed_update = []

#Each row has a list of genres
#See if '(no genres listed)' appears in each row containing a list of genres
for index, rows in duplicate_titles.iterrows():
    genres = []
    for i in range(len(duplicate_titles['updated_genres'][index])):
        genres.append(duplicate_titles['updated_genres'][index][i] == '(no genres listed)')
    no_genres_listed_update.append(sum(genres)*len(genres))
    
#Add 'no_genres_listed_update' into movies_df
duplicate_titles['no_genres_listed_update'] = no_genres_listed_update 

In [23]:
#Show titles with that include '(no genres listed)' and have more than one genre listed
duplicate_titles[duplicate_titles['no_genres_listed_update']>1]

,movieId,duplicate_movieId,title,updated_genres,no_genres_listed_update
4,140633,156686,Another World (2014),"[(no genres listed), Documentary]",2
5,162618,181675,Apparition (2014),"[Horror, (no genres listed), Thriller]",3
6,174701,184893,Ava (2017),"[(no genres listed), Drama]",2
9,130388,138656,Black Field (2009),"[(no genres listed), Romance, Drama]",3
18,185191,192243,Contact (1992),"[Mystery, (no genres listed), Drama, Horror, T...",5
28,146624,160868,Escalation (1968),"[(no genres listed), Comedy]",2
29,177993,180465,Escape Room (2017),"[(no genres listed), Horror]",2
30,62467,175857,Family Life (1971),"[(no genres listed), Drama]",2
42,131556,144748,Inside (2012),"[(no genres listed), Horror]",2
47,179783,191867,Let There Be Light (2017),"[(no genres listed), Documentary]",2


In [24]:
#Remove first instance of '(no genres listed)'
for i in duplicate_titles.index:
    if duplicate_titles['no_genres_listed_update'][i]>1: 
        duplicate_titles['updated_genres'][i].remove('(no genres listed)')

In [25]:
#Check these titles successfully removed '(no genres listed)'
duplicate_titles[duplicate_titles['no_genres_listed_update']>1]

,movieId,duplicate_movieId,title,updated_genres,no_genres_listed_update
4,140633,156686,Another World (2014),[Documentary],2
5,162618,181675,Apparition (2014),"[Horror, Thriller]",3
6,174701,184893,Ava (2017),[Drama],2
9,130388,138656,Black Field (2009),"[Romance, Drama]",3
18,185191,192243,Contact (1992),"[Mystery, Drama, Horror, Thriller]",5
28,146624,160868,Escalation (1968),[Comedy],2
29,177993,180465,Escape Room (2017),[Horror],2
30,62467,175857,Family Life (1971),[Drama],2
42,131556,144748,Inside (2012),[Horror],2
47,179783,191867,Let There Be Light (2017),[Documentary],2


In [26]:
#Drop 'no_genres_listed_update' from dataframe
duplicate_titles.drop('no_genres_listed_update',axis=1,inplace=True)

In [27]:
#Check duplicate_titles
duplicate_titles.head()

,movieId,duplicate_movieId,title,updated_genres
0,102190,114130,"20,000 Leagues Under the Sea (1997)","[Sci-Fi, Romance, Adventure]"
1,136696,151797,Absolution (2015),"[Adventure, Crime, Drama, Action, Thriller]"
2,98485,114040,Aftermath (2012),"[Action, Drama, Thriller]"
3,588,114240,Aladdin (1992),"[Adventure, Animation, Fantasy, Comedy, Childr..."
4,140633,156686,Another World (2014),[Documentary]


In [28]:
#Create copy of 'duplicate_titles'
duplicate_titles_movies = duplicate_titles.copy()
#'duplicate_titles_movies': keep movieId and updated genres
duplicate_titles_movies = duplicate_titles_movies.drop(['duplicate_movieId','title'],axis=1)

In [29]:
#Create 'movies_df': copy of 'movies' as dataframe for further use
movies_df = movies.copy()

#Remove duplicates:
movies_df = movies[~movies['title'].duplicated()]
#Check Shape of 'movies_df'
print(movies_df.shape[0])
#'movies' has 58098 rows, now 'movies_df' has 58020 rows, which means 78 duplicates less than the original
#'movies_df' has successfully removed duplicates (58098 - 58020 = 78)

#Update genres:
#Merge to add 'updated_genres' from 'duplicate_titles_movies'
movies_df = movies_df.merge(duplicate_titles_movies, how='left', on = 'movieId')
#Fillna of 'updated_genres' with 'genres'
movies_df['updated_genres'].fillna(movies_df['genres'],inplace=True)

#Complete update of 'genres' by removing 'genres' and renaming 'updated_genres' as 'genres': 
#Collection of column names from 'movies'
movies_df_columns = movies.columns
#Drop 'genres'
movies_df.drop('genres', axis=1, inplace=True)
#Rename columns of 'movies_df'
movies_df.columns = movies_df_columns

#Preview 
movies_df

58020


,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
58015,193876,The Great Glinka (1946),[(no genres listed)]
58016,193878,Les tribulations d'une caissière (2011),[Comedy]
58017,193880,Her Name Was Mumu (2016),[Drama]
58018,193882,Flora (2017),"[Adventure, Drama, Horror, Sci-Fi]"


#### Clean 'Title' column
Separate title and year in 'title' into different columns

In [30]:
#Separate year from 'title' column and placed into a new column 'year'

#Create 'year' column from extracting title 
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Remove parentheses in 'year' column
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing '(year)' in 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Apply the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

#Preview
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [34]:
#Convert years from string to integers
movies_df['year'] = pd.to_numeric(movies_df['year'])
movies_df['year'] = movies_df['year'].astype('Int64')

In [35]:
#Check shape of 'movies_df'
movies_df.shape

(58020, 4)

### Genome Scores, Ratings and Tags dataframes

Update movieIds in the same manner for 'genome_scores', 'ratings' and 'tags' dataframes:
1. Create copies of each dataframe - update copies and leave original dataframes only for reference 
2. Update dataframes by converting movieIds from duplicate movie titles to those in the updated 'movies' dataframe
3. Drop timestamps in the 'ratings' and 'tags' dataframes
4. Fill missing data with '' in 'tags' dataframe.

In [36]:
#Make a copies  of dataframes to update
genome_scores_df = genome_scores.copy() 
ratings_df = ratings.copy() 
tags_df = tags.copy()

In [37]:
#Create 'duplicate_titles_update'
duplicate_titles_update = duplicate_titles.drop(['title','updated_genres'], axis=1)
duplicate_titles_update.columns = ['updated_movieId','movieId']

#Preview of 'duplicate_titles_update'
duplicate_titles_update.head()

,updated_movieId,movieId
0,102190,114130
1,136696,151797
2,98485,114040
3,588,114240
4,140633,156686


In [38]:
#Update the following dataframes by movieId
ratings_df = update_df(ratings_df)
tags_df = update_df(tags_df)

In [39]:
#Remove timestamp from 'ratings_df' and 'tags_df'
ratings_df.drop('timestamp',axis=1,inplace=True)
tags_df.drop('timestamp',axis=1,inplace=True)

In [40]:
#In 'tags_df', find NaN tags
tags_df[tags_df.isnull().any(axis=1)]

,userId,movieId,tag
483263,80439,123,NaN
483264,80439,346,NaN
483268,80439,1184,NaN
483275,80439,1785,NaN
483276,80439,2194,NaN
483278,80439,2691,NaN
483286,80439,4103,NaN
483288,80439,4473,NaN
483290,80439,4616,NaN
483306,80439,7624,NaN


In [41]:
#In 'tags' fill NaN with ''
tags_df.fillna('',inplace=True)

#Check if NaNs have been filled
tags_df.isnull().sum()

userId     0
movieId    0
tag        0
dtype: int64